In [1]:
import pandas as pd
import random
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os

%config InlineBackend.figure_format = 'retina'

In [2]:
# Download the dataset once
if not os.path.exists('fakenet_dataset'):
    !aws s3 cp s3://danylo-ucla/fakenet_dataset.zip ./
    !unzip fakenet_dataset.zip >/dev/null

In [3]:
!tree -L 2 fakenet_dataset

/bin/sh: tree: command not found


## Training Images and Metadata

In [4]:
train_metadata = pd.read_csv('fakenet_dataset/train/metadata.csv')

In [5]:
train_metadata.head()

file_name_training                   file_name_generated  \
0  ffd603965b2eb89bfa6e7bda19c903f9.jpg  d70026e562baeb2976d33ea72bbe2af2.jpg   
1  c5e15bda1013a6a8c28b87d23aa94767.jpg  41c055de712d74957659ab5d901c4212.jpg   
2  517547ba7695808a52ecd2e2f4b6faf7.jpg  aab1106eb9793b08b665902c74c8e0cf.jpg   
3  c3ff13909de96ed069fb2769800ea4b9.jpg  0658524e5dc4058fe7ceff0a72dbda18.jpg   
4  2b5c6c916a7110287f7193081906cc5e.jpg  8546adea5c43ea5e517abfa5208aa10d.jpg   

                                             caption  
0  a toilet with a green and white door and green...  
1  An older couple embracing each other next to a...  
2  A square glazed doughnut, and a chocolate doug...  
3   A large bathroom with tiled floor and big mirror  
4     A man riding a snowboard down a conveyor belt.

In [6]:
random.seed(43)

In [7]:
i = random.randint(0, train_metadata.shape[0])

file_name_training = train_metadata['file_name_training'].iloc[i]
file_name_generated = train_metadata['file_name_generated'].iloc[i]
caption = train_metadata['caption'].iloc[i]

plt.suptitle(caption)

plt.subplot(1, 2, 1)
plt.title('Original')
plt.imshow(Image.open(f'fakenet_dataset/train/images/{file_name_training}'))
plt.xticks([])
plt.yticks([])

plt.subplot(1, 2, 2)
plt.title('Generated')
plt.imshow(Image.open(f'fakenet_dataset/train/images/{file_name_generated}'))
plt.xticks([])
plt.yticks([])

([], [])

In [8]:
#train and val split
from sklearn.model_selection import train_test_split
train_metadata_train, train_metadata_val = train_test_split(train_metadata, test_size=0.1, random_state=43)

## Score Images and Metadata

In [9]:
score_metadata = pd.read_csv('fakenet_dataset/score/metadata.csv')
score_metadata

file_name_a  \
0     7ad84509c9b0226bc0288537a8ab22ae.jpg   
1     a4a39968e428b3c82257e25920e3adeb.jpg   
2     38ffec577b2c097f3a3403ae6c6638b8.jpg   
3     07ecbbbc269ba1bd2d5b792f9abea0ed.jpg   
4     10c585ad69ed895ac1a3b58d5dfb256a.jpg   
...                                    ...   
2811  1541d399a4be2e0c85ef061e6bddb60c.jpg   
2812  d1bc62f2b725409f46421219f16383df.jpg   
2813  7015aa30ddb1d1605c6822d2856e98db.jpg   
2814  3b76a717ad707699851465a0680f5798.jpg   
2815  048699161f7cd04b8dd6ecb601fcbf17.jpg   

                               file_name_b  \
0     798d20e113a6bde4525336ce66a84a69.jpg   
1     2d6c056b045062c97450a29dd83d8c63.jpg   
2     050d2c637cbfa509f9a0072d6ca422b5.jpg   
3     23cbc59d33f061d2930f27c641a01807.jpg   
4     7bbe5ae59f098437099f5ba093c2b38b.jpg   
...                                    ...   
2811  15e819f3f3ede6e2e20170d71185031d.jpg   
2812  06f2ad1c20ed610523d677e3cbb8fed6.jpg   
2813  846bd374ab91c40386fa48c1f5fdef78.jpg   
2814  f5a5c27c396aae7142805befc710f9ac.jpg   
2815  168fa67e8614b1008a084763049cf865.jpg   

                                                caption  
0     A couple of motorcycles parked next to each ot...  
1     A vase with a bird on it and a matching winegl...  
2                   a man that is surfing on some water  
3     A crowd of young people sanding on a snow cove...  
4     A kitchen with a refrigerator, counter and was...  
...                                                 ...  
2811      a close up of a person wearing a suit and tie  
2812  Zebras eat from a trough set up in their zoo e...  
2813       A group of people on a field with a Frisbee.  
2814  A boy sitting at a table eating food with food...  
2815  A man riding a snowboard down a snow covered s...  

[2816 rows x 3 columns]

## Creating Tensorflow Dataset (Single Image, Labeled Segment)

In [24]:
# See Python Generator
# https://peps.python.org/pep-0255/
def build_generator_labeled_single(metadata: pd.DataFrame):
    def generator():
        for _, row in metadata.iterrows():
            training_path = 'fakenet_dataset/train/images/' + row['file_name_training']
            generated_path = 'fakenet_dataset/train/images/' + row['file_name_generated']

            training_np = np.array(Image.open(training_path)).astype(np.float32)
            generated_np = np.array(Image.open(generated_path)).astype(np.float32)

            model_input = training_np
            model_output = (1,)
            yield (model_input, model_output)

            model_input = generated_np
            model_output = (0,)
            yield (model_input, model_output)
    return generator


# See Tensorflow Dataset
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
def build_dataset_labeled_single(metadata: pd.DataFrame) -> tf.data.Dataset:
    model_input = tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32)  # type: ignore
    model_output = tf.TensorSpec(shape=(1,), dtype=tf.int32)  # type: ignore

    dataset_signature = (model_input, model_output)

    dataset = tf.data.Dataset.from_generator(
        build_generator_labeled_single(metadata), 
        output_signature=dataset_signature
    )

    return dataset


In [25]:
train_single_dataset = build_dataset_labeled_single(train_metadata)
model_input, model_output = next(iter(train_single_dataset))
display(model_input.shape, model_output.shape)

2022-10-22 22:03:23.506458: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 22:03:30.527277: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 22:03:30.528296: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 22:03:30.661254: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

TensorShape([224, 224, 3])

TensorShape([1])

In [26]:
train_single_dataset = train_single_dataset.batch(16)
model_input, model_output = next(iter(train_single_dataset))
display(model_input.shape, model_output.shape)

TensorShape([16, 224, 224, 3])

TensorShape([16, 1])

In [41]:
model_output

<tf.Tensor: shape=(16, 1), dtype=int32, numpy=
array([[1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]], dtype=int32)>

## Creating Tensorflow Dataset (Single Image, Score Segment)

In [27]:
import tensorflow as tf
import numpy as np

# See Python Generator
# https://peps.python.org/pep-0255/
def generator_score_single():
    metadata = pd.read_csv('fakenet_dataset/score/metadata.csv')
    
    for _, row in metadata.iterrows():
        image_a_path = 'fakenet_dataset/score/images/' + row['file_name_a']
        image_b_path = 'fakenet_dataset/score/images/' + row['file_name_b']

        image_a_np = np.array(Image.open(image_a_path))
        image_b_np = np.array(Image.open(image_b_path))

        model_input = image_a_np
        yield model_input

        # Uncomment if you want to compare the two images after inference
        # model_input = image_b_np
        # yield model_input


# See Tensorflow Dataset
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
def build_dataset_score_single() -> tf.data.Dataset:
    model_input = tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32)  # type: ignore

    dataset_signature = model_input

    dataset = tf.data.Dataset.from_generator(
        generator_score_single, 
        output_signature=dataset_signature
    )

    return dataset

In [28]:
score_single_dataset = build_dataset_score_single().batch(16)
model_input = next(iter(score_single_dataset))
display(model_input.shape)

TensorShape([16, 224, 224, 3])

In [42]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile('sgd', 'binary_crossentropy','accuracy')
model.fit(train_single_dataset)
score_y_hat = model.predict(score_single_dataset)

1408/1408 [==============================] - 44s 31ms/step - loss: 2070043.7500 - accuracy: 0.5128


## Creating Tensorflow Dataset (Paired Images, Labeled Segment)

In [10]:
# See Python Generator
# https://peps.python.org/pep-0255/
def build_generator_labeled_paired(metadata: pd.DataFrame):
    def generator():
        for _, row in metadata.iterrows():
            training_path = 'fakenet_dataset/train/images/' + row['file_name_training']
            generated_path = 'fakenet_dataset/train/images/' + row['file_name_generated']

            training_np = np.array(Image.open(training_path))
            generated_np = np.array(Image.open(generated_path))

            model_output = np.random.randint(low=0, high=2, size=1)

            if model_output == 1:
                model_input = (training_np, generated_np)
            else:
                model_input = (generated_np, training_np)

            yield (model_input, model_output)

    return generator

# See Tensorflow Dataset
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
def build_dataset_labeled_paired(metadata: pd.DataFrame):
    image_signature = tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32)  # type: ignore

    model_input = (image_signature, image_signature)
    model_output = tf.TensorSpec(shape=(1,), dtype=tf.int32)  # type: ignore

    dataset_signature = (model_input, model_output)

    dataset = tf.data.Dataset.from_generator(
        build_generator_labeled_paired(metadata), 
        output_signature=dataset_signature
    )

    return dataset

In [11]:
train_paired_dataset_train = build_dataset_labeled_paired(train_metadata_train).batch(16)
train_paired_dataset_val = build_dataset_labeled_paired(train_metadata_val).batch(16)

(model_input_a, model_input_b), model_output = next(iter(train_paired_dataset_train))

display(model_input_a.shape, model_input_b.shape, model_output.shape)

2022-10-23 17:20:18.710660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 17:20:25.312281: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 17:20:25.313214: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 17:20:25.437764: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

TensorShape([16, 224, 224, 3])

TensorShape([16, 224, 224, 3])

TensorShape([16, 1])

In [12]:
# See Python Generator
# https://peps.python.org/pep-0255/
def generator_score_paired():
    metadata = pd.read_csv('fakenet_dataset/score/metadata.csv')

    for _, row in metadata.iterrows():
        image_a_path = 'fakenet_dataset/score/images/' + row['file_name_a']
        image_b_path = 'fakenet_dataset/score/images/' + row['file_name_b']

        image_a_np = np.array(Image.open(image_a_path))
        image_b_np = np.array(Image.open(image_b_path))

        model_input = (image_a_np, image_b_np)
        yield (model_input,)


# See Tensorflow Dataset
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
def build_dataset_score_paired():
    image_signature = tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32)  # type: ignore

    model_input = (image_signature, image_signature)

    dataset_signature = (model_input,)

    dataset = tf.data.Dataset.from_generator(
        generator_score_paired, 
        output_signature=dataset_signature
    )

    return dataset

In [13]:
score_paired_dataset = build_dataset_score_paired().batch(16)

((model_input_a, model_input_b),) = next(iter(score_paired_dataset))

display(model_input_a.shape, model_input_b.shape)

TensorShape([16, 224, 224, 3])

TensorShape([16, 224, 224, 3])

In [14]:
# https://arxiv.org/pdf/1801.04381.pdf
# https://keras.io/api/applications/mobilenet/#mobilenetv2-function
base_model = tf.keras.applications.MobileNetV3Small(
    include_top=False,
    input_shape=(224, 224, 3),
    weights="imagenet")

4349952/4334752 [==============================] - 0s 0us/step


In [15]:
# Freezes the parameters of the MobileNet layers, so they will not update during training.
# These parameters are initialized to a pre-trained snapshot using the ImagetNet dataset.
base_model.trainable = False

In [16]:
# See Keras Functional API
# https://keras.io/guides/functional_api/

inputs = [
    tf.keras.layers.Input((224, 224, 3)),
    tf.keras.layers.Input((224, 224, 3)),
]

# Example of reusing the same layer twice.
# A Model object (e.g. a pre-trained model) can also be used multiple times.

#layer = tf.keras.applications.mobilenet_v2.preprocess_input
#outputs = [
    #layer(inputs[0]),
    #layer(inputs[1]),
#]

base = base_model
outputs = [
    base(inputs[0]),
    base(inputs[1]),
]


outputs = tf.keras.layers.Concatenate(axis=-1)(outputs)
outputs = tf.keras.layers.GlobalAveragePooling2D()(outputs)
outputs = tf.keras.layers.Flatten()(outputs)

outputs = tf.keras.layers.BatchNormalization()(outputs)
outputs = tf.keras.layers.LeakyReLU()(outputs)
outputs = tf.keras.layers.Dense(1024,activation='relu', activity_regularizer=tf.keras.regularizers.L1(0.0001))(outputs)
outputs = tf.keras.layers.Dense(1024,activation='relu')(outputs)
outputs = tf.keras.layers.Dense(512,activation='relu')(outputs)
outputs = tf.keras.layers.Dropout(0.2, seed=43)(outputs)
#outputs = tf.keras.layers.BatchNormalization()(outputs)
#outputs = tf.keras.layers.LeakyReLU()(outputs)

#outputs = tf.keras.layers.Flatten()(outputs)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(outputs)

model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 MobilenetV3small (Functional)  (None, 7, 7, 576)    939120      ['input_2[0][0]',                
                                                                  'input_3[0][0]']            

In [18]:
model.compile(optimizer = tf.keras.optimizers.Adam(lr=0.001),
              loss = 'binary_crossentropy', 
              metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy",threshold=0.5)])

model.fit(train_paired_dataset_train, validation_data=train_paired_dataset_val, batch_size = 128*2, epochs = 10, callbacks=[
        tf.keras.callbacks.TerminateOnNaN(),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',   
            min_delta=0.1,
            patience=3,
            cooldown=10,
            verbose=1
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',   
            min_delta=0.1,
            patience=5
        )
    ])

/home/ec2-user/SageMaker/setup/miniconda/envs/ucla_deeplearning/lib/python3.10/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10


2022-10-23 17:23:20.077050: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8401
2022-10-23 17:24:37.292615: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.0.145, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


634/634 [==============================] - 314s 218ms/step - loss: 0.3158 - binary_accuracy: 0.8756 - val_loss: 0.1941 - val_binary_accuracy: 0.9255 - lr: 0.0010
Epoch 2/10
634/634 [==============================] - 71s 112ms/step - loss: 0.2069 - binary_accuracy: 0.9283 - val_loss: 0.2074 - val_binary_accuracy: 0.9308 - lr: 0.0010
Epoch 3/10
634/634 [==============================] - 70s 110ms/step - loss: 0.1699 - binary_accuracy: 0.9415 - val_loss: 0.1687 - val_binary_accuracy: 0.9352 - lr: 0.0010
Epoch 4/10
634/634 [==============================] - ETA: 0s - loss: 0.1543 - binary_accuracy: 0.9504
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
634/634 [==============================] - 71s 113ms/step - loss: 0.1543 - binary_accuracy: 0.9504 - val_loss: 0.1889 - val_binary_accuracy: 0.9397 - lr: 0.0010
Epoch 5/10
634/634 [==============================] - 70s 111ms/step - loss: 0.0939 - binary_accuracy: 0.9687 - val_loss: 0.1352 - val_binary_accuracy: 0

In [19]:
#fine tuning
base_model.trainable = True

In [20]:
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [21]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(lr=0.001/10),
    #optimizer=tf.keras.optimizers.Adam(lr=0.0001),
    loss='binary_crossentropy',
    metrics=[
        tf.keras.metrics.BinaryAccuracy(
            name="binary_accuracy",
            threshold=0.5
        )
    ]
)

/home/ec2-user/SageMaker/setup/miniconda/envs/ucla_deeplearning/lib/python3.10/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [22]:
initial_epochs = 10
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_paired_dataset_train,
                         epochs=total_epochs,
                         initial_epoch=10,
                         batch_size=128,
                         validation_data=train_paired_dataset_val,
                        callbacks=[
        tf.keras.callbacks.TerminateOnNaN(),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',   
            min_delta=0.1,
            patience=3,
            cooldown=10,
            verbose=1
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',   
            min_delta=0.1,
            patience=5
        )
    ])

Epoch 11/20
634/634 [==============================] - 118s 135ms/step - loss: 0.0997 - binary_accuracy: 0.9681 - val_loss: 0.1121 - val_binary_accuracy: 0.9698 - lr: 1.0000e-04
Epoch 12/20
634/634 [==============================] - 84s 132ms/step - loss: 0.0614 - binary_accuracy: 0.9830 - val_loss: 0.0861 - val_binary_accuracy: 0.9814 - lr: 1.0000e-04
Epoch 13/20
634/634 [==============================] - 84s 132ms/step - loss: 0.0490 - binary_accuracy: 0.9871 - val_loss: 0.0865 - val_binary_accuracy: 0.9849 - lr: 1.0000e-04
Epoch 14/20
634/634 [==============================] - ETA: 0s - loss: 0.0356 - binary_accuracy: 0.9916
Epoch 14: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
634/634 [==============================] - 85s 134ms/step - loss: 0.0356 - binary_accuracy: 0.9916 - val_loss: 0.0705 - val_binary_accuracy: 0.9885 - lr: 1.0000e-04
Epoch 15/20
634/634 [==============================] - 84s 132ms/step - loss: 0.0357 - binary_accuracy: 0.9919 - val_loss:

In [23]:
score_y_hat = model.predict(score_paired_dataset)

In [24]:
# The expected model output for the score segment is a DataFrame with 1 column -- 'probability'.
# One probability per row in the score metadata.
# The probability represents the likelihood that the image in the 'file_name_a' column is real.

# The following code can be used to check the DataFrame in 'score_y_hat':

score_y_hat = pd.DataFrame(score_y_hat, columns=['probability'])
assert score_y_hat.shape == (score_metadata.shape[0], 1)
assert score_y_hat.probability.dtype == np.float32
assert score_y_hat.probability.min() >= 0
assert score_y_hat.probability.max() <= 1

In [25]:
# Once you are satisfied with your model,
# your need to submit 4 files:
# 1. this notebook and/or any other Python scripts
# 2. Model definition in JSON format
# 3. Model parameters in H5 format
# 4. the probabilities for the score segment

import os
model_dir = 'fakenet_model'
os.makedirs(model_dir, exist_ok=True)

# Save model definition:
import json
with open(f'{model_dir}/keras_model.json', 'w') as f:
    model_json = json.dumps(json.loads(model.to_json()), indent=True)
    f.write(model_json)

# Save the learned parameters.
model.save_weights(f"{model_dir}/keras_parameters.h5")

# Finally, let's save the forecast.
score_y_hat.to_parquet(f'{model_dir}/score_y_hat.parquet')

In [28]:
score_y_hat.round(5)

probability
0             0.0
1             1.0
2             1.0
3             1.0
4             1.0
...           ...
2811          1.0
2812          1.0
2813          1.0
2814          1.0
2815          0.0

[2816 rows x 1 columns]